In [1]:
import json
import geopandas as gpd
import glob
import pandas as pd
import dask_geopandas

# Calculate Canopy Cover by Dissemination Area

In [2]:
# Reprojection
# NAD83 / UTM zone 11N

area_proj = "EPSG:26911"

## Dissemination Area Boundaries

In [3]:
# #https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-eng.cfm?year=21

In [3]:
DA = gpd.read_file("../../data/DisseminationArea/lda_000b21a_e.shp")

In [4]:
census_data = pd.read_csv("DA_Calgary_census_data.csv")
df = DA.merge(census_data, left_on="DGUID", right_on="REF_AREA").drop(
    ["DAUID", "LANDAREA", "PRUID", "REF_AREA"], axis=1
)
df = df.to_crs("EPSG:4326")
df["geom_reproj"] = df.to_crs(area_proj).geometry

In [5]:
dask_df = dask_geopandas.from_geopandas(df, npartitions=4)

## Canopy Data


In [6]:
# Calculate canopy covery for each community boundary


def calc_canopy(x, file):
    # "../../Trees_2022/geo_export_dc81e6e2-f4a7-4c85-8a9c-deabecc1015d.shp",
    print(f"DGUID: {x.DGUID}")
    df1 = gpd.read_file(file, mask=x.geometry)
    df1_clipped = df1.clip(x.geometry)

    # df = df.to_crs("EPSG:4326")

    df1_clipped = df1_clipped.to_crs(area_proj)
    tree_area = df1_clipped.area.sum()
    return tree_area / x.geom_reproj.area


meta_df = pd.DataFrame(columns=["frac"], dtype="float64")

In [7]:
%%time
shape_files = glob.glob("../../data/shapefile/2020/*.shp")
for file in shape_files:
    print(f"Processing {file}")

    year = file.split("/")[-2]

    df["frac"] = dask_df.apply(
        lambda x: calc_canopy(x, file), axis=1, meta=meta_df
    ).compute()
    df[["frac", "geometry", "DGUID"]].to_file(
        f"canopy_{year}_da.json", driver="GeoJSON"
    )

    print(f"Saved canopy_{year}.json")

Processing ../../data/shapefile/2020/geo_export_27c5b67c-4340-4dcf-bd1e-b4fd236cf0dc.shp
DGUID: 2021S051248061885
DGUID: 2021S051248061123
DGUID: 2021S051248060056
DGUID: 2021S051248060616
DGUID: 2021S051248061124DGUID: 2021S051248060057
DGUID: 2021S051248061886

DGUID: 2021S051248061125
DGUID: 2021S051248060617
DGUID: 2021S051248061887
DGUID: 2021S051248061128
DGUID: 2021S051248060058
DGUID: 2021S051248061888
DGUID: 2021S051248060618DGUID: 2021S051248061131

DGUID: 2021S051248060059
DGUID: 2021S051248061132
DGUID: 2021S051248061946
DGUID: 2021S051248060060
DGUID: 2021S051248061133
DGUID: 2021S051248060619
DGUID: 2021S051248060061DGUID: 2021S051248061134

DGUID: 2021S051248061947
DGUID: 2021S051248060062
DGUID: 2021S051248061135
DGUID: 2021S051248060063
DGUID: 2021S051248060620
DGUID: 2021S051248061949
DGUID: 2021S051248061951
DGUID: 2021S051248061136
DGUID: 2021S051248060064
DGUID: 2021S051248061953DGUID: 2021S051248060621
DGUID: 2021S051248061137

DGUID: 2021S051248060065
DGUID: 2021

In [8]:
# mask = df.iloc[300].geometry
# df1 = gpd.read_file(
#         "../../data/shapefile/2022/geo_export_dc81e6e2-f4a7-4c85-8a9c-deabecc1015d.shp",
#         mask=mask
#     )

# df1_clipped = df1.clip(mask)
# df1_clipped = df1_clipped.to_crs(area_proj)
# tree_area = df1_clipped.area.sum()